# 1. Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.svm import SVR

[07:29:30] Initializing Normalizer


# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC6_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  3083 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

3083

# 3.Standardization SDF file for work set

In [5]:
records_ws = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToMolBlock(all_mols_ws[i])
    records_ws.append(record)
            
mols_ws = []
for i,record in enumerate(records_ws):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ws.append(m)
           
moldf_ws = []
for val in mols_ws:
    if val != None:
        moldf_ws.append(val)
print('Kept data: ', len(moldf_ws), 'molecules')

[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharger
[07:29:47] Running Normalizer
[07:29:47] Running Uncharg

Kept data:  3083 molecules


[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharger
[07:30:07] Running Normalizer
[07:30:07] Running Uncharg

# 4.Data entry and curation test set

In [6]:
uploaded_file_ts="datasets/HDAC6_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  771 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [7]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [8]:
len(y_ts)

771

# 5.Standardization SDF file for test set

In [9]:
records_ts = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToMolBlock(all_mols_ts[i])
    records_ts.append(record)
            
mols_ts = []
for i,record in enumerate(records_ts):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ts.append(m)
           
moldf_ts = []
for val in mols_ts:
    if val != None:
        moldf_ts.append(val)
print('Kept data: ', len(moldf_ts), 'molecules')

[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharger
[07:30:10] Running Normalizer
[07:30:10] Running Uncharg

Kept data:  771 molecules


[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharger
[07:30:16] Running Normalizer
[07:30:16] Running Uncharg

# 6.Calculation MorganFingerprint for work set

In [10]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [11]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [12]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [13]:
savetxt('Models/Morgan_fingerprint/x_tr_MF.csv', x_tr, delimiter=',')

In [14]:
x_tr.shape

(3083, 1024)

# 7.Calculation MorganFingerprint for test set

In [15]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [16]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [17]:
x_ts = rdkit_numpy_convert(fp_ts)

In [18]:
x_ts.shape

(771, 1024)

In [19]:
type(x_tr)

numpy.ndarray

In [20]:
x_tr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [22]:
y_tr

array([ 4.02,  4.03,  4.06, ...,  9.77,  9.85, 10.  ], dtype=float32)

# 8. SVM model building and validation

In [72]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [73]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [74]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=2, cv=cv, verbose=1)

In [75]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [76]:
svm.best_params_
best_svm = svm.best_estimator_

In [77]:
svm.best_params_

{'C': 10, 'gamma': 0.01}

In [91]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [92]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 3)
Q2_CV

0.656

In [28]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.69

# 9. Prediction for test set's molecules

In [115]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [116]:
y_pred_svm = best_svm.predict(x_ts)

In [117]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.68

In [118]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.68

save the model to disk

In [35]:
pickle.dump(best_svm, open('Models/Morgan_fingerprint/HDAC6_SVM_MF.pkl', 'wb'))

load the model from disk

In [87]:
best_svm = pickle.load(open('Models/Morgan_fingerprint/HDAC6_SVM_MF.pkl', 'rb'))

# 10. Y-randomization RF model

In [95]:
permutations = 50
score, permutation_scores, pvalue = permutation_test_score(best_svm, x_tr, y_tr,
                                                           cv=cv, scoring='r2',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=seed)
print('True score = ', score.round(3),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  6.4min


True score =  0.655 
Y-randomization =  -0.31 
p-value =  0.0196


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 10.1min finished


# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [119]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [120]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.162278,5.656854,4.472136,3.872983,6.708204,3.316625,6.244998,5.000000,3.464102,6.480741,...,3.162278,3.605551,4.123106,4.242640,3.162278,3.605551,3.872983,3.741657,2.645751,4.795832
2,6.164414,6.000000,5.385165,4.358899,6.782330,3.872983,6.403124,5.099020,4.242640,6.633250,...,3.162278,5.567764,4.472136,4.358899,3.605551,5.099020,3.872983,4.000000,3.464102,5.477226
3,6.164414,6.082763,5.567764,4.795832,6.855655,4.472136,7.071068,5.099020,4.242640,6.708204,...,4.472136,5.916080,4.690416,4.472136,3.741657,5.291502,4.242640,4.242640,3.464102,5.567764
4,6.244998,6.082763,5.567764,4.795832,6.855655,5.196152,7.141428,5.099020,4.582576,6.708204,...,4.582576,6.000000,4.795832,4.690416,3.872983,5.477226,4.242640,4.242640,3.464102,5.567764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,11.000000,11.090536,11.269427,10.630146,11.313708,10.816654,11.532562,11.401754,11.401754,11.313708,...,11.090536,11.445523,11.401754,11.090536,11.000000,11.313708,11.090536,11.180340,11.135529,10.862781
3079,11.000000,11.090536,11.269427,10.677078,11.313708,10.816654,11.532562,11.401754,11.445523,11.313708,...,11.090536,11.445523,11.401754,11.090536,11.135529,11.313708,11.180340,11.180340,11.135529,11.180340
3080,11.000000,11.090536,11.269427,10.677078,11.313708,10.816654,11.532562,11.401754,11.445523,11.313708,...,11.180340,11.445523,11.401754,11.090536,11.135529,11.313708,11.269427,11.224972,11.269427,11.180340
3081,11.180340,11.269427,11.357817,10.677078,11.401754,11.045361,11.704700,11.489125,11.532562,11.313708,...,11.180340,11.445523,11.401754,11.313708,11.135529,11.313708,11.401754,11.224972,11.313708,11.180340


In [121]:
similarity= neighbors_k

In [122]:
Dmean=np.mean(similarity[1,:])

In [123]:
round(Dmean, 2)

3.23

In [124]:
std=np.std(similarity[1,:])

In [125]:
round(std, 2)

1.37

In [126]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.91


In [127]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [128]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,3.605551,4.358899,3.316625,4.898980,5.000000,1.000000,2.449490,5.000000,2.449490,3.605551,...,3.162278,3.316625,3.872983,3.605551,2.645751,4.582576,3.605551,3.605551,1.000000,4.898980
1,4.898980,4.582576,3.605551,5.099020,5.196152,3.316625,3.464102,5.099020,2.828427,3.872983,...,4.123106,4.582576,4.000000,3.605551,3.605551,6.855655,5.099020,4.690416,3.605551,5.000000
2,5.196152,4.898980,3.605551,6.633250,5.196152,4.123106,3.872983,5.291502,3.464102,4.000000,...,4.582576,4.582576,4.795832,4.123106,3.872983,6.855655,5.196152,4.795832,5.477226,5.099020
3,5.196152,5.196152,4.000000,7.211102,5.291502,4.358899,3.872983,5.291502,4.358899,4.358899,...,4.582576,4.582576,5.567764,4.582576,4.000000,6.855655,5.385165,4.898980,5.477226,5.099020
4,5.477226,5.385165,5.567764,7.211102,5.385165,4.472136,4.123106,5.291502,6.633250,4.898980,...,4.582576,4.690416,6.082763,4.898980,4.242640,6.928203,5.385165,5.099020,5.656854,5.196152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,10.816654,11.224972,11.000000,11.313708,11.180340,11.401754,10.723805,11.489125,11.489125,10.862781,...,11.180340,10.677078,11.357817,11.489125,11.045361,11.180340,11.269427,11.090536,11.045361,11.090536
3079,10.816654,11.313708,11.401754,11.357817,11.180340,11.401754,11.000000,11.532562,11.532562,10.862781,...,11.224972,10.677078,11.532562,11.575837,11.090536,11.224972,11.269427,11.090536,11.045361,11.135529
3080,10.816654,11.313708,11.401754,11.357817,11.180340,11.401754,11.000000,11.575837,11.532562,10.862781,...,11.224972,10.770329,11.532562,11.789826,11.090536,11.269427,11.269427,11.090536,11.090536,11.135529
3081,10.816654,11.357817,11.401754,11.401754,11.180340,11.401754,11.000000,11.618950,11.532562,11.224972,...,11.224972,10.770329,11.532562,11.789826,11.090536,11.357817,11.269427,11.180340,11.090536,11.180340


In [129]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[3.606 4.359 3.317 4.899 5.    1.    2.449 5.    2.449 3.606 3.464 2.828
 6.928 3.162 0.    4.123 1.    5.    3.162 4.123 2.646 3.162 4.899 4.359
 6.    2.828 3.464 4.583 3.317 4.    3.742 7.    3.606 4.    4.243 3.606
 3.873 3.873 3.317 3.606 4.    1.    3.317 4.472 2.646 4.359 2.828 2.828
 3.162 3.464 3.873 3.464 5.916 3.162 4.359 1.732 3.    3.606 6.481 3.742
 3.606 3.742 3.464 2.828 5.099 4.123 3.162 3.464 3.    3.606 3.742 3.162
 1.    4.899 4.472 3.464 3.    4.472 6.856 1.    3.606 4.359 3.742 3.606
 5.831 3.464 4.359 2.    4.69  1.    2.    4.69  3.742 4.69  2.646 4.583
 3.742 3.162 1.414 3.464 1.414 6.481 0.    4.    2.646 4.359 2.828 3.873
 3.162 3.464 1.    3.606 4.359 3.    3.464 3.317 2.449 2.236 3.317 1.
 3.606 2.236 0.    4.243 4.243 3.742 3.464 4.243 4.583 2.449 3.873 0.
 2.828 2.449 3.873 1.732 5.099 3.464 0.    2.449 3.317 3.    3.606 0.
 3.    4.    0.    2.    3.606 1.414 3.742 1.414 4.    5.477 2.828 4.
 4.123 4.    4.899 0.    3.742 3.    3.873 3.162 3.464 0.    2.

In [130]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False  True False False  True  True False  True  True  True  True
 False  True  True False  True False  True False  True  True False False
 False  True  True False  True False  True False  True False False  True
  True  True  True  True False  True  True False  True False  True  True
  True  True  True  True False  True False  True  True  True False  True
  True  True  True  True False False  True  True  True  True  True  True
  True False False  True  True False False  True  True False  True  True
 False  True False  True False  True  True False  True False  True False
  True  True  True  True  True False  True False  True False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True False False  True  True False False  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True False False  True False
 False False False  True  True  True  True  True  T

In [131]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.72


In [132]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   2   5   6   8   9  10  11  13  14  16  18  20  21  25  26  28  30
  32  35  36  37  38  39  41  42  44  46  47  48  49  50  51  53  55  56
  57  59  60  61  62  63  66  67  68  69  70  71  72  75  76  79  80  82
  83  85  87  89  90  92  94  96  97  98  99 100 102 104 106 107 108 109
 110 111 113 114 115 116 117 118 119 120 121 122 125 126 129 130 131 132
 133 134 135 137 138 139 140 141 142 143 144 146 147 148 149 150 151 154
 159 160 161 162 163 164 165 166 168 169 170 172 174 175 176 181 182 183
 184 185 187 188 189 190 191 193 195 196 197 198 200 201 202 203 204 205
 206 207 208 209 210 211 212 214 217 218 219 220 221 223 224 225 226 228
 230 231 232 233 237 238 239 240 241 244 245 246 248 249 250 251 252 253
 254 255 256 257 258 259 261 262 264 265 266 268 270 271 273 274 275 276
 278 279 280 281 282 284 285 286 288 289 290 291 292 293 294 295 296 297
 298 299 300 302 305 306 309 310 311 312 314 315 316 317 320 321 324 327
 328 329 33

In [133]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [134]:
y_pred_svm_ad=list(y_pred_svm)

In [135]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [136]:
len(y_pred_svm_ad)

553

In [137]:
y_ts_ad=list(y_ts)

In [138]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [139]:
len(y_ts_ad)

553

In [140]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.73

In [141]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.64